## Load splits.json into a dictionary

In [ ]:
import json
from pathlib import Path

splits_path = Path('splits.json')
with splits_path.open('r', encoding='utf-8') as f:
    splits_dict = json.load(f)

splits_dict